In [37]:
import pandas as pd
import pyarrow.parquet as pq
import os
import airportsdata 
from geopy.distance import geodesic

import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import pycountry
import pycountry_convert as pc
import plotly.io as pio

import folium
from folium.plugins import FastMarkerCluster
from folium.plugins import HeatMap

In [38]:
airports = airportsdata.load()

dataset = pq.ParquetDataset('c:\\Users\\lukas\\OneDrive - Imperial College London\\0_Sustainable Transport\\2021_COP26',
                            ) # filters=[('columnName', 'in', filterList)]
df = dataset.read(use_threads=True).to_pandas()

In [39]:
# Exclude private jets
private_jet = ['CL60','P180','GLF5','GLEX','GLF6','F2TH','C17','LJ60','F900','GLF4','E35L','CL35','C30J',
               'GL5T','ASTR','C130','BE20','H25B','L410','F100','CL30','LJ35','PC24','C56X','E55P','PAY3']
df = df[~df['aircraft_type_icao'].isin(private_jet)]

In [40]:
df_dubai = df[df['destination_airport']== 'OMDB']

# Group by 'origin_airport' and select the first occurrence of each group
unique_origins = df_dubai.groupby('origin_airport').first().reset_index()
unique_origins = unique_origins[['origin_airport','longitude_first_waypoint','latitude_first_waypoint']]
unique_origins.to_excel('unique_destinations_globally.xlsx', index=False)

In [42]:
delegates_path = os.getcwd() + '/data/origin_airports_delegates.csv'
data_delegates = pd.read_csv(delegates_path)

iata_icao_path = os.getcwd() + '/data/iata-icao.csv'
data_iata_icao = pd.read_csv(iata_icao_path)

In [44]:
airport_location = data_delegates.merge(data_iata_icao, on='iata', how='left')[['iata','icao','total_delegates','country_code','airport','latitude','longitude']]


In [45]:
# Check for NAN values 
# airport_location.isna()
# airport_location = airport_location.dropna()#isna().sum()#.any(axis=1).count
# airport_location[airport_location.isna().any(axis=1)]
airport_location["country_code"][77] =  "NA"

In [46]:
dubai_airport = pd.DataFrame({'latitude_A': [airports.get('OMDB')['lat']],
                     'longitude_A': [airports.get('OMDB')['lon']]})

delegate_airports = airport_location[['latitude','longitude']]

# Function to calculate distance using geopy.distance
def calculate_distance(row):
    coord_A = (dubai_airport['latitude_A'].iloc[0], dubai_airport['longitude_A'].iloc[0])
    coord_B = (row['latitude'], row['longitude'])
    return geodesic(coord_A, coord_B).kilometers

airport_location['distance_to_dubai'] = delegate_airports.apply(calculate_distance,axis=1)

airport_location

,iata,icao,total_delegates,country_code,airport,latitude,longitude,distance_to_dubai
0,AUH,OMAA,4409,AE,Abu Dhabi International Airport,24.43300,54.65110,115.950398
1,GRU,SBGR,7598,BR,Sao Paulo/Guarulhos International Airport,-23.42620,-46.48000,12218.280093
2,PEK,ZBAA,3480,CN,Beijing Capital International Airport,40.07250,116.59800,5856.254969
3,LOS,DNMM,1411,NG,Murtala Muhammed International Airport,6.57737,3.32116,5899.623220
4,CGK,WIII,1229,ID,Soekarno–Hatta International Airport,-6.12556,106.65600,6543.695620
...,...,...,...,...,...,...,...,...
202,ZRH,LSZH,8,CH,Zurich Airport,47.46470,8.54917,4773.283197
203,KIV,LUKK,8,MD,Chisinau International Airport,46.92770,28.93100,3352.541593
204,ASM,HHAS,7,ER,Asmara International Airport,15.29190,38.91070,2039.020269
205,MGA,MNMG,6,NI,Augusto C. Sandino International Airport,12.14150,-86.16820,14143.820362


In [47]:
airport_location[airport_location['distance_to_dubai'] >= 10000]

,iata,icao,total_delegates,country_code,airport,latitude,longitude,distance_to_dubai
1,GRU,SBGR,7598,BR,Sao Paulo/Guarulhos International Airport,-23.426200,-46.4800,12218.280093
10,JFK,KJFK,1899,US,John F. Kennedy International Airport,40.639700,-73.7789,11021.839311
13,YOW,CYOW,1830,CA,Ottawa Macdonald-Cartier International Airport,45.322500,-75.6692,10756.579780
32,SYD,YSSY,996,AU,Sydney Airport (Kingsford Smith Airport),-33.946100,151.1770,12038.861443
37,BOG,SKBO,932,CO,El Dorado International Airport,4.701590,-74.1469,13645.133727
45,SCL,SCEL,742,CL,Comodoro Arturo Merino Benitez International A...,-33.393000,-70.7858,14776.568263
46,SCL,SCEL,742,CL,Arturo Merino Benítez International Airport,-33.392800,-70.7856,14776.547899
80,LIM,SPIM,195,PE,Jorge Chavez International Airport,-12.021900,-77.1143,14836.731110
90,AEP,SABE,179,AR,Aeroparque Internacional Jorge Newbery,-34.558900,-58.4164,13659.050933
105,TGU,MHTG,139,HN,Toncontin International Airport,14.060900,-87.2172,14069.337152


In [48]:
print(airport_location[airport_location['distance_to_dubai'] >= 10000]['distance_to_dubai'].count(),"flights with over 10,000 km")

55 flights with over 10,000 km


In [49]:
print("Total km flown to COP28:",round(sum(airport_location['distance_to_dubai']),0))
print("Total Delegates:          ",sum(airport_location.total_delegates))

Total km flown to COP28: 1401384.0
Total Delegates:           81939


# Calculating CO2 emissions with take-off and landing (non-transit)

In [50]:
def country_code_to_name(country_code):
    try:
        return pycountry.countries.get(alpha_2=country_code).name
    except AttributeError:
        return None

plot_data = airport_location[:][:40]
plot_data
plot_data['country_name'] = plot_data['country_code'].apply(country_code_to_name)

In [51]:
plot_data.loc[0,"country_name"] = 'UAE'
plot_data.loc[12,"country_name"] = 'Tanzania'
plot_data.loc[17,"country_name"] = 'Korea'
plot_data.loc[20,"country_name"] = 'Congo'
plot_data.loc[21,"country_name"] = 'Russia'

In [52]:
# # Plot the data
# plt.figure(figsize=(20, 6))  # Set the figure size
# plt.bar(plot_data['country_name'], plot_data.total_delegates)  # Plot a bar chart
# plt.title('Participants from each Country at COP28')  # Set the title
# plt.xlabel('Country')  # Set the x-axis label
# plt.ylabel('# participants')  # Set the y-axis label
# plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
# plt.tight_layout()  # Adjust layout to prevent clipping of labels
# plt.savefig('participants_country.pdf')
# plt.show()  # Show the plot

sorted_indices = sorted(range(len(plot_data['total_delegates'])), key=lambda i: plot_data['total_delegates'][i])
sorted_categories = [plot_data['country_name'][i] for i in sorted_indices]
sorted_values = [plot_data['total_delegates'][i] for i in sorted_indices]

fig = go.Figure(data=[go.Bar(
    y=sorted_categories,  # y-axis categories
    x=sorted_values,      # x-axis values
    orientation='h'  # horizontal orientation
)])

# Add title and labels
fig.update_layout(
    title='Participants from each country (Top 40)',
    yaxis=dict(title='Countries'),
    xaxis=dict(title='# participants'),
    width=400,
    height=800
)

# Show the chart
#fig.write_image("participants_country.png")
fig.show()
#pio.write_image(fig, "participants_country.png")


### Checking if flight from origin to destination is in flight database

In [103]:
destination_dubai = df[df['destination_airport'] == 'OMDB']
# destination_dubai[destination_dubai['origin_airport'] == 'DNMM'] #[['origin_airport_name','destination_airport_name']]
# destination_dubai['origin_airport'] = destination_dubai['origin_airport'].astype('str')#.dtypes
print(len(destination_dubai))
destination_dubai.iloc[:3]


23328


,callsign,icao_address,flight_number,tail_number,aircraft_type_icao,aircraft_engine_type,origin_airport,origin_airport_name,origin_country,destination_airport,...,so2_kg,sulphates_kg,nvpm_data_source,nvpm_mass_kg,nvpm_number,nvpm_ei_n_mean,sdr_mean,night_flight,night_segments,pct_night_dist
93,FDB1916,8963B5,FZ1916,A6-FES,B738,Jet,URML,Makhachkala Uytash International Airport,RU,OMDB,...,8.002414,0.163315,ICAO EDB,0.363317,7.447394e+18,1.116772e+15,14.200313,False,True,74.495262
213,FDB1462,8963B1,FZ1462,A6-FEO,B738,Jet,UCFM,Manas International Airport,KG,OMDB,...,11.051537,0.225542,ICAO EDB,0.491998,1.079065e+19,1.171672e+15,90.088696,False,True,23.943959
1194,FDB448,896331,FZ448,A6-FDZ,B738,Jet,VOMM,Chennai International Airport,IN,OMDB,...,10.664273,0.217638,ICAO EDB,0.453730,9.797427e+18,1.102458e+15,122.496717,False,False,0.000000


In [104]:
# # Alternative 
# def inner_merge_on_columns(df1, df2, column1, column2):
#     merged_data = []
#     for value1 in df1[column1]:
#         for value2 in df2[column2]:
#             if value1 == value2:
#                 row1 = df1[df1[column1] == value1].iloc[0].to_dict()
#                 row2 = df2[df2[column2] == value2].iloc[0].to_dict()
#                 merged_data.append({**row1, **row2})
#     return pd.DataFrame(merged_data)

# merged_df = inner_merge_on_columns(airport_location,destination_dubai,"icao","origin_airport")
# merged_df['icao'].unique()


# Merging airport origin with the existing flights from 2021
merged_df = airport_location.merge(destination_dubai, left_on='icao', right_on='origin_airport', how='inner')
test = merged_df['icao'].unique()
print("Flights who have a direct flight to XX:",test.size)

Flights who have a direct flight to XX: 87


##### Select unique flights from origin to destination airport

In [105]:
merged_df = merged_df.sort_values(by=['icao', 'co2_kg'])
unique_flights_df = merged_df.drop_duplicates(subset=['icao'])
unique_flights_df.reset_index(inplace=True)

# Relevant columns
unique_flights_df = unique_flights_df[['icao', 'total_delegates', 'country_code', 'origin_airport','destination_airport',
       'distance_to_dubai', 'assumed_load_factor', 'aircraft_type_icao', 'total_fuel_burn','co2_kg', 
       'nox_kg','hc_kg', 'oc_kg', 'h2o_kg', 'so2_kg', 'sulphates_kg', 'latitude','longitude'
       ]]
unique_flights_df.to_excel('flights_without_transit.xlsx', index=False)
unique_flights_df.iloc[:3]

,icao,total_delegates,country_code,origin_airport,destination_airport,distance_to_dubai,assumed_load_factor,aircraft_type_icao,total_fuel_burn,co2_kg,nox_kg,hc_kg,oc_kg,h2o_kg,so2_kg,sulphates_kg,latitude,longitude
0,DAAG,33,DZ,DAAG,OMDB,5076.841504,0.625348,A332,28026.934111,88537.084858,377.908845,2.433629,0.560539,34473.128957,33.632321,0.686374,36.69100,3.215410
1,DGAA,618,GH,DGAA,OMDB,6292.961752,0.623285,B737,16668.516582,52655.843883,181.799396,1.341575,0.333370,20502.275396,20.002220,0.408209,5.60519,-0.166786
2,DTTA,221,TN,DTTA,OMDB,4451.568920,0.619923,B77L,30416.465762,96085.615343,554.819782,6.042459,0.608329,37412.252888,36.499759,0.744893,36.85100,10.227200


In [106]:
unique_flights_df['aircraft_type_icao'].unique()

array(['A332', 'B737', 'B77L', 'B38M', 'A319', 'B738', 'A20N', 'B77W',
       'BCS3', 'B733', 'B734', 'B39M', 'A333', 'A388', 'B789', 'FA7X',
       'CRJ2', 'A19N', 'A320', 'E75S', 'A21N', 'DH8D', 'G280', 'A359'],
      dtype=object)

#### Flights without direct flight to dubai

In [107]:
# All origins who 
airport_location['existing_flight'] = airport_location['icao'].isin(merged_df['origin_airport'])
# airport_location[airport_location['existing_flight'] == False]

In [101]:
non_direct = airport_location[airport_location['existing_flight'] == False]
# test = pd.merge(export, data_iata_icao, on='icao', how='left')
# test.drop_duplicates(subset=['icao'])
# test.to_excel('flights_with_transit_stop_with_latitude.xlsx', index=False)
non_direct.to_excel('non_existing_flights.xlsx', index=False)
non_direct.reset_index(inplace=True)
non_direct = non_direct[['iata','icao','total_delegates','country_code','airport','latitude','longitude','distance_to_dubai','existing_flight']]
non_direct.iloc[:3]

,iata,icao,total_delegates,country_code,airport,latitude,longitude,distance_to_dubai,existing_flight
0,LOS,DNMM,1411,NG,Murtala Muhammed International Airport,6.57737,3.32116,5899.623220,False
1,RBA,GMME,823,MA,Rabat-Sale Airport,34.05150,-6.75152,6003.076078,False
2,YOW,CYOW,1830,CA,Ottawa Macdonald-Cartier International Airport,45.32250,-75.66920,10756.579780,False


## Calculate non-direct flights with stop-over


In [133]:
path_transit = os.getcwd() + '/data/transit_flight_stopover.csv'
df_stopover = pd.read_csv(path_transit)
print(len(df_stopover))
df_stopover.iloc[:3]

114


,iata,icao,first_closest,second_closest,third_closest,to_dubai
0,LOS,DNMM,DGAA,FNLU,GOBD,OMDB
1,RBA,GMME,GMMN,LPPT,LEMD,OMDB
2,YOW,CYOW,CYYZ,KEWR,KJFK,OMDB


#### First_closest
##### Origin -> hub (First_closest)

In [134]:
merged_df = pd.merge(df_stopover, df, left_on=['icao', 'first_closest'], right_on=['origin_airport', 'destination_airport'], how='inner')
merged_df = merged_df.drop_duplicates(subset=['icao'], keep='first')
merged_df.reset_index(inplace=True)
merged_df = merged_df.rename(columns={'destination_airport':'hub_airport'})
merged_df = merged_df[['icao','first_closest','origin_airport','hub_airport',
       'assumed_load_factor', 'aircraft_type_icao', 'total_fuel_burn','co2_kg', 
       'nox_kg','hc_kg', 'oc_kg', 'h2o_kg', 'so2_kg', 'sulphates_kg','to_dubai'
       ]]
print(len(merged_df))
merged_df.iloc[:3]

56


,icao,first_closest,origin_airport,hub_airport,assumed_load_factor,aircraft_type_icao,total_fuel_burn,co2_kg,nox_kg,hc_kg,oc_kg,h2o_kg,so2_kg,sulphates_kg,to_dubai
0,DNMM,DGAA,DNMM,DGAA,0.607831,A332,4349.248039,13739.274554,91.839270,0.886448,0.086985,5349.575088,5.219098,0.106512,OMDB
1,GMME,GMMN,GMME,GMMN,0.628559,E190,298.382863,942.591464,2.627597,0.489429,0.005968,367.010921,0.358059,0.007307,OMDB
2,CYOW,CYYZ,CYOW,CYYZ,0.751742,A320,1577.426378,4983.089929,21.963573,0.399045,0.031549,1940.234445,1.892912,0.038631,OMDB


In [148]:
merged_df['hub_airport'].unique()

array(['DGAA', 'GMMN', 'CYYZ', 'FNLU', 'GOBD', 'KMIA', 'SBGR', 'FIMP',
       'FAOR', 'DTTA', 'EFHK', 'KIAH', 'YSSY', 'EIDW', 'VVTS', 'YPDN',
       'YBBN', 'LICC', 'VTBD', 'EVRA', 'LBSF'], dtype=object)

##### Hub -> Dubai (First_closest)

In [136]:
# Remaining flights to be found! 
occurrences_count = merged_df['origin_airport'].value_counts().to_dict()

# Delete the same number of occurrences from df_stopover['icao'] for each value in final_merged_df['origin_airport']
for value, count in occurrences_count.items():
    df_stopover = df_stopover.drop(df_stopover[df_stopover['icao'] == value].head(count).index)

len(df_stopover)

58

In [147]:
results_closest = merged_df['hub_airport','to_dubai']
destination_dubai = pd.merge(merged_df, df, left_on=['hub_airport', 'to_dubai'], right_on=['origin_airport', 'destination_airport'], how='inner')
# destination_dubai = destination_dubai.drop_duplicates(subset=['icao'], keep='first')
destination_dubai.reset_index(inplace=True)
# destination_dubai = destination_dubai[['icao','first_closest','origin_airport','destination_airport',
#        'assumed_load_factor', 'aircraft_type_icao', 'total_fuel_burn','co2_kg', 
#        'nox_kg','hc_kg', 'oc_kg', 'h2o_kg', 'so2_kg', 'sulphates_kg','to_dubai'
#        ]]
print(len(destination_dubai))
destination_dubai.iloc[:3]
# destination_dubai['hub_airport'].unique()

2466


,index,icao,first_closest,origin_airport_x,hub_airport,assumed_load_factor_x,aircraft_type_icao_x,total_fuel_burn_x,co2_kg_x,nox_kg_x,...,so2_kg_y,sulphates_kg_y,nvpm_data_source,nvpm_mass_kg,nvpm_number,nvpm_ei_n_mean,sdr_mean,night_flight,night_segments,pct_night_dist
0,0,DNMM,DGAA,DNMM,DGAA,0.607831,A332,4349.248039,13739.274554,91.83927,...,64.838105,1.323227,ICAO EDB,0.782432,1.579042e+19,2.922434e+14,5.303684,False,True,87.065310
1,1,DNMM,DGAA,DNMM,DGAA,0.607831,A332,4349.248039,13739.274554,91.83927,...,64.274027,1.311715,ICAO EDB,0.774759,1.571920e+19,2.934784e+14,3.525180,False,True,89.861495
2,2,DNMM,DGAA,DNMM,DGAA,0.607831,A332,4349.248039,13739.274554,91.83927,...,62.228377,1.269967,ICAO EDB,0.750300,1.520234e+19,2.931590e+14,4.772261,False,True,87.562822


In [143]:
filtered_destinations = destination_dubai[(destination_dubai['hub_airport'] == 'SBGR') & (destination_dubai['to_dubai'] == 'OMDB')]
# merged_result = pd.merge(combined_df, filtered_destinations, how='left', left_index=True, right_index=True)
filtered_destinations

,index,icao,first_closest,origin_airport_x,hub_airport,assumed_load_factor_x,aircraft_type_icao_x,total_fuel_burn_x,co2_kg_x,nox_kg_x,...,so2_kg_y,sulphates_kg_y,nvpm_data_source,nvpm_mass_kg,nvpm_number,nvpm_ei_n_mean,sdr_mean,night_flight,night_segments,pct_night_dist
0,0,SCEL,SBGR,SCEL,SBGR,0.787394,B788,13642.567129,43096.869562,329.315423,...,140.433602,2.865992,ICAO EDB,1.696263,3.403254e+19,2.908068e+14,506.236026,False,True,32.579686
1,1,SCEL,SBGR,SCEL,SBGR,0.787394,B788,13642.567129,43096.869562,329.315423,...,142.172976,2.901489,ICAO EDB,1.720208,3.421815e+19,2.888156e+14,503.741589,False,True,31.300598
2,2,SCEL,SBGR,SCEL,SBGR,0.787394,B788,13642.567129,43096.869562,329.315423,...,141.971114,2.897370,ICAO EDB,1.716742,3.424086e+19,2.894183e+14,505.600147,False,True,33.528884
3,3,SCEL,SBGR,SCEL,SBGR,0.787394,B788,13642.567129,43096.869562,329.315423,...,141.811154,2.894105,ICAO EDB,1.716282,3.408507e+19,2.884264e+14,502.450860,False,True,33.200428
4,4,SCEL,SBGR,SCEL,SBGR,0.787394,B788,13642.567129,43096.869562,329.315423,...,138.480462,2.826132,ICAO EDB,1.671697,3.363291e+19,2.914454e+14,499.994279,False,True,34.032114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,62,SCEL,SBGR,SCEL,SBGR,0.787394,B788,13642.567129,43096.869562,329.315423,...,192.103898,3.920488,ICAO EDB,12.259340,7.927913e+19,4.952266e+14,444.923472,False,True,32.617901
63,63,SCEL,SBGR,SCEL,SBGR,0.787394,B788,13642.567129,43096.869562,329.315423,...,206.032481,4.204745,ICAO EDB,12.736935,7.731577e+19,4.503121e+14,445.420299,False,True,34.203770
64,64,SCEL,SBGR,SCEL,SBGR,0.787394,B788,13642.567129,43096.869562,329.315423,...,201.820904,4.118794,ICAO EDB,12.524242,7.643250e+19,4.544574e+14,438.957780,False,True,33.870773
65,65,SCEL,SBGR,SCEL,SBGR,0.787394,B788,13642.567129,43096.869562,329.315423,...,198.974132,4.060697,ICAO EDB,12.612900,8.347000e+19,5.034021e+14,425.997940,False,True,34.154942


In [ ]:

combined_df = pd.concat([merged_df, stop_over_dubai_closest], ignore_index=True)
combined_df

In [578]:
my_list = []

# OLD APPROACH

##### Check if flights from (stopover -> dubai) exist

In [579]:
# indicator_list = pd.DataFrame()
destination_dubai
stop_over_dubai = df_stopover.merge(destination_dubai, left_on='Closest airport', right_on='origin_airport', how='inner') 
stop_over_dubai = stop_over_dubai.drop_duplicates(subset=['indicator'])
stop_over_dubai.reset_index(inplace=True)
stop_over_dubai = stop_over_dubai[['indicator','Closest airport','origin_airport','destination_airport',
       'assumed_load_factor', 'aircraft_type_icao', 'total_fuel_burn','co2_kg', 
       'nox_kg','hc_kg', 'oc_kg', 'h2o_kg', 'so2_kg', 'sulphates_kg',
       ]]
# stop_over_dubai

,indicator,Closest airport,origin_airport,destination_airport,assumed_load_factor,aircraft_type_icao,total_fuel_burn,co2_kg,nox_kg,hc_kg,oc_kg,h2o_kg,so2_kg,sulphates_kg
0,1,DGAA,DGAA,OMDB,0.606079,B77W,54031.754570,170686.312686,1138.533461,6.055622,1.080635,66459.058121,64.838105,1.323227
1,5,DGAA,DGAA,OMDB,0.606079,B77W,54031.754570,170686.312686,1138.533461,6.055622,1.080635,66459.058121,64.838105,1.323227
2,6,DGAA,DGAA,OMDB,0.606079,B77W,54031.754570,170686.312686,1138.533461,6.055622,1.080635,66459.058121,64.838105,1.323227
3,7,DGAA,DGAA,OMDB,0.606079,B77W,54031.754570,170686.312686,1138.533461,6.055622,1.080635,66459.058121,64.838105,1.323227
4,13,DGAA,DGAA,OMDB,0.606079,B77W,54031.754570,170686.312686,1138.533461,6.055622,1.080635,66459.058121,64.838105,1.323227
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,104,LATI,LATI,OMDB,0.698774,B38M,8783.294863,27746.428472,131.278370,0.988350,0.175666,10803.452681,10.539954,0.215101
110,110,LEMD,LEMD,OMDB,0.698774,B77W,46417.394145,146632.548105,965.073362,6.189943,0.928348,57093.394799,55.700873,1.136753
111,111,LEMD,LEMD,OMDB,0.698774,B77W,46417.394145,146632.548105,965.073362,6.189943,0.928348,57093.394799,55.700873,1.136753
112,112,LROP,LROP,OMDB,0.698774,B38M,8147.357014,25737.500808,120.042470,0.863296,0.162947,10021.249127,9.776828,0.199527


In [503]:
# my_list += test.indicator.tolist()
# unique_list = list(set(my_list))
# unique_list
# pd.DataFrame(unique_list)

##### Closest airport (origin -> stopover)

In [594]:
# How much more space for business class? 1.5
path_seats = os.getcwd() + '/data/Airplane_data.xlsx'
df_airplane_seats = pd.read_excel(path_seats)
# df_airplane_seats

In [595]:
non_direct.iloc[:3]

,iata,icao,total_delegates,country_code,airport,latitude,longitude,distance_to_dubai,existing_flight
0,LOS,DNMM,1411,NG,Murtala Muhammed International Airport,6.57737,3.32116,5899.623220,False
1,RBA,GMME,823,MA,Rabat-Sale Airport,34.05150,-6.75152,6003.076078,False
2,YOW,CYOW,1830,CA,Ottawa Macdonald-Cartier International Airport,45.32250,-75.66920,10756.579780,False


In [596]:
# Only look at flights including passenger seats
# values_to_drop = df_airplane_seats['Airplane type']
# df_limited = df[df['aircraft_type_icao'].isin(values_to_drop)]
# df_limited#.iloc[:3]

In [614]:
# List to store merged DataFrames
merged_dfs = []

# Alternative 1
# Iterate over the 'Closest airport' column
for dest in df_stopover['Closest airport']:
    # Perform the merge operation
    merged_result = non_direct.merge(df[df['destination_airport'] == dest], left_on='icao', right_on='origin_airport', how='inner')
    
    # Check if merged result is not empty
    if not merged_result.empty:
        # Append to the list
        merged_dfs.append(merged_result)

# Alternative 2
# merged_dfs = [non_direct.merge(df[df['destination_airport'] == dest], left_on='icao', right_on='origin_airport', how='inner') for dest in df_stopover['Closest airport']]

# Concatenate the list of merged DataFrames into a single DataFrame
final_merged_df = pd.concat(merged_dfs, ignore_index=True)
# final_merged_df = final_merged_df.merge(df_stopover,left_on='destination_airport',right_on='Closest airport', how='outer')

# Save the final merged DataFrame
#final_merged_df.to_csv('merged_data.csv', index=False)  # You can change the filename and format as needed


# final_merged_df = final_merged_df.sort_values(by=['icao', 'co2_kg'])
final_merged_df = final_merged_df.drop_duplicates(subset=['icao'], keep='first')
final_merged_df.reset_index(inplace=True)

In [640]:
final_merged_df = final_merged_df[['icao','total_delegates','country_code','origin_airport','destination_airport',
       'assumed_load_factor', 'aircraft_type_icao', 'total_fuel_burn','co2_kg', 
       'nox_kg','hc_kg', 'oc_kg', 'h2o_kg', 'so2_kg', 'sulphates_kg',
       ]]
final_merged_df.iloc[:5]

,icao,total_delegates,country_code,origin_airport,destination_airport,assumed_load_factor,aircraft_type_icao,total_fuel_burn,co2_kg,nox_kg,hc_kg,oc_kg,h2o_kg,so2_kg,sulphates_kg
0,DNMM,1411,NG,DNMM,DGAA,0.607831,A332,4349.248039,13739.274554,91.839270,0.886448,0.086985,5349.575088,5.219098,0.106512
1,DIAP,401,CI,DIAP,DGAA,0.606079,B77W,5064.038696,15997.298239,152.825187,2.379407,0.101281,6228.767596,6.076846,0.124017
2,SCEL,742,CL,SCEL,DGAA,0.806921,B742,103503.977023,326969.063415,2527.802413,86.860188,2.070080,127309.891738,124.204772,2.534791
3,DXXX,250,TG,DXXX,DGAA,0.606079,B737,960.517712,3034.275452,11.243859,0.191087,0.019210,1181.436786,1.152621,0.023523
4,GBYD,238,GM,GBYD,DGAA,0.623088,B737,8342.230739,26353.106904,85.949498,1.202897,0.166845,10260.943809,10.010677,0.204300


In [641]:
len(final_merged_df['origin_airport'])

59

In [642]:
final_merged_df['aircraft_type_icao'].unique()

array(['A332', 'B77W', 'B742', 'B737', 'B738', 'A21N', 'E190', 'A320',
       'A339', 'A20N', 'A319', 'B763', 'B788', 'A306', 'A321', 'B38M',
       'A333', 'A124', 'AT76', 'A359', 'CRJ2', 'B733', 'D228', 'MA60'],
      dtype=object)

Add flights from stopover --> dubai

In [89]:

# Count the occurrences of each value in final_merged_df['origin_airport']
occurrences_count = final_merged_df['origin_airport'].value_counts().to_dict()

# Delete the same number of occurrences from non_direct['icao'] for each value in final_merged_df['origin_airport']
for value, count in occurrences_count.items():
    non_direct = non_direct.drop(non_direct[non_direct['icao'] == value].head(count).index)

len(non_direct)

NameError: name 'final_merged_df' is not defined

In [550]:
# Select flights from destination to dubai 
values_to_drop = final_merged_df['destination_airport']#[82,37,38]

stop_over_dubai_closest = stop_over_dubai[stop_over_dubai['origin_airport'].isin(values_to_drop)]
stop_over_dubai_closest#['origin_airport']


,indicator,Closest airport,origin_airport,destination_airport,assumed_load_factor,aircraft_type_icao,total_fuel_burn,co2_kg,nox_kg,hc_kg,oc_kg,h2o_kg,so2_kg,sulphates_kg
0,1,DGAA,DGAA,OMDB,0.606079,B77W,54031.754570,170686.312686,1138.533461,6.055622,1.080635,66459.058121,64.838105,1.323227
1,5,DGAA,DGAA,OMDB,0.606079,B77W,54031.754570,170686.312686,1138.533461,6.055622,1.080635,66459.058121,64.838105,1.323227
2,6,DGAA,DGAA,OMDB,0.606079,B77W,54031.754570,170686.312686,1138.533461,6.055622,1.080635,66459.058121,64.838105,1.323227
3,7,DGAA,DGAA,OMDB,0.606079,B77W,54031.754570,170686.312686,1138.533461,6.055622,1.080635,66459.058121,64.838105,1.323227
4,13,DGAA,DGAA,OMDB,0.606079,B77W,54031.754570,170686.312686,1138.533461,6.055622,1.080635,66459.058121,64.838105,1.323227
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,91,EVRA,EVRA,OMDB,0.698774,BCS3,12562.649595,39685.410069,186.390360,0.047850,0.251253,15452.059001,15.075180,0.307657
107,100,LBSF,LBSF,OMDB,0.698774,B38M,8439.818710,26661.387304,121.903167,0.901405,0.168796,10380.977013,10.127782,0.206689
110,110,LEMD,LEMD,OMDB,0.698774,B77W,46417.394145,146632.548105,965.073362,6.189943,0.928348,57093.394799,55.700873,1.136753
111,111,LEMD,LEMD,OMDB,0.698774,B77W,46417.394145,146632.548105,965.073362,6.189943,0.928348,57093.394799,55.700873,1.136753


In [511]:
combined_df = pd.concat([final_merged_df, stop_over_dubai_closest], ignore_index=True)
combined_df

,iata,total_delegates,country_code,origin_airport,destination_airport,assumed_load_factor,aircraft_type_icao,total_fuel_burn,co2_kg,nox_kg,hc_kg,oc_kg,h2o_kg,so2_kg,sulphates_kg,indicator,Closest airport
0,LOS,1411.0,NG,DNMM,DGAA,0.607831,A332,4349.248039,13739.274554,91.839270,0.886448,0.086985,5349.575088,5.219098,0.106512,NaN,NaN
1,ABJ,401.0,CI,DIAP,DGAA,0.606079,B77W,5064.038696,15997.298239,152.825187,2.379407,0.101281,6228.767596,6.076846,0.124017,NaN,NaN
2,SCL,742.0,CL,SCEL,DGAA,0.806921,B742,103503.977023,326969.063415,2527.802413,86.860188,2.070080,127309.891738,124.204772,2.534791,NaN,NaN
3,LFW,250.0,TG,DXXX,DGAA,0.606079,B737,960.517712,3034.275452,11.243859,0.191087,0.019210,1181.436786,1.152621,0.023523,NaN,NaN
4,BJL,238.0,GM,GBYD,DGAA,0.623088,B737,8342.230739,26353.106904,85.949498,1.202897,0.166845,10260.943809,10.010677,0.204300,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,NaN,NaN,NaN,LATI,OMDB,0.698774,B38M,8783.294863,27746.428472,131.278370,0.988350,0.175666,10803.452681,10.539954,0.215101,104.0,LATI
166,NaN,NaN,NaN,LEMD,OMDB,0.698774,B77W,46417.394145,146632.548105,965.073362,6.189943,0.928348,57093.394799,55.700873,1.136753,110.0,LEMD
167,NaN,NaN,NaN,LEMD,OMDB,0.698774,B77W,46417.394145,146632.548105,965.073362,6.189943,0.928348,57093.394799,55.700873,1.136753,111.0,LEMD
168,NaN,NaN,NaN,LROP,OMDB,0.698774,B38M,8147.357014,25737.500808,120.042470,0.863296,0.162947,10021.249127,9.776828,0.199527,112.0,LROP


##### 2nd Closest Airport

In [512]:
# df_stopover

In [513]:
# Mssing 37,38,82 (-1)
df_stopover_two = df_stopover.iloc[[36,37,81],:]

In [647]:
for dest in df_stopover_two['2nd closest']:
    # Perform the merge operation
    merged_result = non_direct.merge(df[df['destination_airport'] == dest], left_on='icao', right_on='origin_airport', how='inner')
    
    # Check if merged result is not empty
    if not merged_result.empty:
        # Append to the list
        merged_dfs.append(merged_result)
# merged_dfs = [non_direct.merge(df[df['destination_airport'] == dest], left_on='icao', right_on='origin_airport', how='inner') for dest in df_stopover_two['2nd closest']]

final_merged_df = pd.concat(merged_dfs, ignore_index=True)
# final_merged_df = final_merged_df.merge(df_stopover_two,left_on='icao',right_on='2nd closest', how='inner')

# final_merged_df = final_merged_df.sort_values(by=['icao', 'co2_kg'])
final_merged_df = final_merged_df.drop_duplicates(subset=['icao'])
final_merged_df.reset_index(inplace=True)
final_merged_df = final_merged_df[[	'iata','total_delegates','country_code','origin_airport','destination_airport',
       'assumed_load_factor', 'aircraft_type_icao', 'total_fuel_burn','co2_kg', 
       'nox_kg','hc_kg', 'oc_kg', 'h2o_kg', 'so2_kg', 'sulphates_kg',
       ]]
final_merged_df

,iata,total_delegates,country_code,origin_airport,destination_airport,assumed_load_factor,aircraft_type_icao,total_fuel_burn,co2_kg,nox_kg,hc_kg,oc_kg,h2o_kg,so2_kg,sulphates_kg
0,LOS,1411,NG,DNMM,DGAA,0.607831,A332,4349.248039,13739.274554,91.839270,0.886448,0.086985,5349.575088,5.219098,0.106512
1,ABJ,401,CI,DIAP,DGAA,0.606079,B77W,5064.038696,15997.298239,152.825187,2.379407,0.101281,6228.767596,6.076846,0.124017
2,SCL,742,CL,SCEL,DGAA,0.806921,B742,103503.977023,326969.063415,2527.802413,86.860188,2.070080,127309.891738,124.204772,2.534791
3,LFW,250,TG,DXXX,DGAA,0.606079,B737,960.517712,3034.275452,11.243859,0.191087,0.019210,1181.436786,1.152621,0.023523
4,BJL,238,GM,GBYD,DGAA,0.623088,B737,8342.230739,26353.106904,85.949498,1.202897,0.166845,10260.943809,10.010677,0.204300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,LLW,305,MW,FWKI,HAAB,0.607831,B738,7120.243351,22492.848746,78.971934,0.875419,0.142405,8757.899322,8.544292,0.174373
61,COO,297,BJ,DBBB,HAAB,0.606079,B738,11829.294786,37368.742227,132.447511,0.906710,0.236586,14550.032586,14.195154,0.289697
62,BZV,284,CG,FCBB,HAAB,0.608707,B738,8796.717621,27788.830964,91.281349,0.866205,0.175934,10819.962673,10.556061,0.215430
63,WDH,212,NaN,FYWH,HAAB,0.606079,B788,20560.379081,64950.237517,433.523507,0.014201,0.411208,25289.266270,24.672455,0.503519


In [515]:
values_to_keep = [82,37,38]

stop_over_dubai_2ndclosest = stop_over_dubai[stop_over_dubai['indicator'].isin(values_to_keep)]
stop_over_dubai_2ndclosest

,indicator,Closest airport,origin_airport,destination_airport,assumed_load_factor,aircraft_type_icao,total_fuel_burn,co2_kg,nox_kg,hc_kg,oc_kg,h2o_kg,so2_kg,sulphates_kg
81,37,HDAM,HDAM,OMDB,0.607831,B738,6753.901206,21335.573911,78.125123,0.771647,0.135078,8307.298484,8.104681,0.165402
82,38,HDAM,HDAM,OMDB,0.607831,B738,6753.901206,21335.573911,78.125123,0.771647,0.135078,8307.298484,8.104681,0.165402
103,82,HHAS,HHAS,OMDB,0.606955,B738,6400.667852,20219.709743,74.459191,0.782703,0.128013,7872.821458,7.680801,0.156751


In [516]:
combined_df = pd.concat([combined_df, stop_over_dubai_2ndclosest], ignore_index=True)
combined_df = pd.concat([combined_df, final_merged_df], ignore_index=True)
combined_df


,iata,total_delegates,country_code,origin_airport,destination_airport,assumed_load_factor,aircraft_type_icao,total_fuel_burn,co2_kg,nox_kg,hc_kg,oc_kg,h2o_kg,so2_kg,sulphates_kg,indicator,Closest airport
0,LOS,1411.0,NG,DNMM,DGAA,0.607831,A332,4349.248039,13739.274554,91.839270,0.886448,0.086985,5349.575088,5.219098,0.106512,NaN,NaN
1,ABJ,401.0,CI,DIAP,DGAA,0.606079,B77W,5064.038696,15997.298239,152.825187,2.379407,0.101281,6228.767596,6.076846,0.124017,NaN,NaN
2,SCL,742.0,CL,SCEL,DGAA,0.806921,B742,103503.977023,326969.063415,2527.802413,86.860188,2.070080,127309.891738,124.204772,2.534791,NaN,NaN
3,LFW,250.0,TG,DXXX,DGAA,0.606079,B737,960.517712,3034.275452,11.243859,0.191087,0.019210,1181.436786,1.152621,0.023523,NaN,NaN
4,BJL,238.0,GM,GBYD,DGAA,0.623088,B737,8342.230739,26353.106904,85.949498,1.202897,0.166845,10260.943809,10.010677,0.204300,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,SVD,15.0,VC,TVSA,KJFK,0.806407,B738,10247.764582,32372.688314,117.455933,0.901605,0.204955,12604.750436,12.297317,0.250966,NaN,NaN
234,IEV,180.0,UA,UKKK,EVRA,0.654094,A320,3466.281558,10949.983441,46.656895,0.207006,0.069326,4263.526316,4.159538,0.084889,NaN,NaN
235,VTE,40.0,LA,VLVT,VVNB,0.654302,MA60,822.830654,2599.322035,12.457656,0.427872,0.016457,1012.081704,0.987397,0.020151,NaN,NaN
236,BWN,84.0,BN,WBSB,RPLL,0.653054,A20N,3492.157949,11031.726962,49.461449,0.199587,0.069843,4295.354278,4.190590,0.085522,NaN,NaN


In [517]:
combined_df[combined_df['aircraft_type_icao'] == 'C208'] 

,iata,total_delegates,country_code,origin_airport,destination_airport,assumed_load_factor,aircraft_type_icao,total_fuel_burn,co2_kg,nox_kg,hc_kg,oc_kg,h2o_kg,so2_kg,sulphates_kg,indicator,Closest airport


In [518]:
combined_df.to_excel('flights_with_transit_stop.xlsx', index=False)

##### Split between economy and business class

## Heatmap

In [520]:
airport_location

,iata,icao,total_delegates,country_code,airport,latitude,longitude,distance_to_dubai,existing_flight
0,AUH,OMAA,4409,AE,Abu Dhabi International Airport,24.43300,54.65110,115.950398,True
1,GRU,SBGR,7598,BR,Sao Paulo/Guarulhos International Airport,-23.42620,-46.48000,12218.280093,True
2,PEK,ZBAA,3480,CN,Beijing Capital International Airport,40.07250,116.59800,5856.254969,True
3,LOS,DNMM,1411,NG,Murtala Muhammed International Airport,6.57737,3.32116,5899.623220,False
4,CGK,WIII,1229,ID,Soekarno–Hatta International Airport,-6.12556,106.65600,6543.695620,True
...,...,...,...,...,...,...,...,...,...
202,ZRH,LSZH,8,CH,Zurich Airport,47.46470,8.54917,4773.283197,True
203,KIV,LUKK,8,MD,Chisinau International Airport,46.92770,28.93100,3352.541593,False
204,ASM,HHAS,7,ER,Asmara International Airport,15.29190,38.91070,2039.020269,True
205,MGA,MNMG,6,NI,Augusto C. Sandino International Airport,12.14150,-86.16820,14143.820362,False


In [521]:
scatter_geo = go.Scattergeo(
    locationmode='USA-states',  # Set location mode
    lon=airport_location['longitude'],
    lat=airport_location['latitude'],
    # text=airport_location[''],
    mode='markers',
    marker=dict(
        size=airport_location['total_delegates']/100,
        # #color=airport_location['total_delegates']/100,
        # colorscale='Viridis',  # Choose a color scale
        # colorbar=dict(title='Values'),  # Add color bar
        line_width=0.5
    )
)

# Create the layout for the figure
layout = go.Layout(
    title='Global Heatmap with Locations and Size Factor',
    geo=dict(
        scope='world',
        showland=True,
        landcolor='rgb(217, 217, 217)',  # Set the color of land areas
        showframe=False,  # Hide the frame
        projection_type='mercator'  # Choose projection type
    ),
    height=800,  # Increase the height of the plot
)

# Create the figure object
fig = go.Figure(data=[scatter_geo], layout=layout)

# Show the figure
fig.show()

In [522]:
# Sample data
locations = airport_location['country_code']
latitudes = airport_location['latitude']
longitudes = airport_location['longitude']
sizes = airport_location['total_delegates']/10
#values = [20, 30, 25, 40]  # Values for the color scale

# Initialize the map centered around the first location
mymap = folium.Map()

# Iterate over locations
for lat, lon, size, location in zip(latitudes, longitudes, sizes, locations):
    # Add circle marker with varying sizes
    folium.CircleMarker(
        location=[lat, lon],
        radius=size / 10,  # Normalize size for better visualization
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        popup=location
    ).add_to(mymap)

# Save the map to an HTML file
mymap.save("map_with_circles.html")

In [523]:
# basemap = folium.Map()
# FastMarkerCluster(airport_location[['latitude', 'longitude']]).add_to(basemap)
# HeatMap(airport_location[['latitude', 'longitude','total_delegates']],
#         ).add_to(basemap)
# basemap

basemap = folium.Map()
FastMarkerCluster(airport_location[['latitude', 'longitude']]).add_to(basemap)
HeatMap(airport_location[['latitude', 'longitude','total_delegates']],
        max_val=max(airport_location['total_delegates'])).add_to(basemap)
basemap

C:\Users\lukas\AppData\Local\Temp\ipykernel_19292\2891524790.py:9: UserWarning:

The `max_val` parameter is no longer necessary. The largest intensity is calculated automatically.

